In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/pokemon/pokemon.csv")
df.head()

In [ ]:
print(sum(df["is_legendary"]==1))
print(sum(df["is_legendary"]==0))

In [ ]:
    percent_missing = df.isnull().sum()*100/len(df)
    missing_value_df = pd.DataFrame({'percent_missing':percent_missing})
    
    missing_value_df

In [ ]:
mean_value=df['height_m'].mean()
df['height_m']=df['height_m'].fillna(mean_value)
mean_value=df['percentage_male'].mean()
df['percentage_male']=df['percentage_male'].fillna(mean_value)
df['type2']=df['type2'].fillna("No")
mean_value=df['weight_kg'].mean()
df['weight_kg']=df['weight_kg'].fillna(mean_value)

In [ ]:
#df.head()

In [ ]:
from ast import literal_eval
#print(len(literal_eval(df["abilities"][800])))

In [ ]:
max_len = 0
j=0
for i in df["abilities"]:
    if max_len<len(literal_eval(df["abilities"][j])):
        max_len = len(literal_eval(df["abilities"][j]))
    df["abilities"][j] = literal_eval(df["abilities"][j])
    j=j+1
print(max_len)

In [ ]:
df = df.join(pd.DataFrame(df.abilities.values.tolist()).add_prefix('abilities_'))
df.head()
print(df["abilities_1"][14])

In [ ]:
df['abilities_1']=df['abilities_1'].fillna("No")
df['abilities_2']=df['abilities_2'].fillna("No")
df['abilities_3']=df['abilities_3'].fillna("No")
df['abilities_4']=df['abilities_4'].fillna("No")
df['abilities_5']=df['abilities_5'].fillna("No")

In [ ]:
df2 = df.drop(columns = ['abilities','japanese_name','name'])
fig = plt.figure(figsize=(15,12))
plt.suptitle("Histograms of numerical columns",fontsize=20)
for i in range(df2.shape[1]):
    plt.subplot(25,2,i+1)
    f = plt.gca()
    f.set_title(df2.columns.values[i])
    vals = np.size(df2.iloc[:,i].unique())
    if vals>=100:
        vals=100
    plt.hist(df2.iloc[:,i],bins=vals, color="#3F5D7D")
plt.tight_layout(rect=[0,0.3,1,0.95])

In [ ]:
df2.corrwith(df.is_legendary).plot.bar(figsize=(20,10),title="correlation of legendary",fontsize=20,rot=45,grid=True,color=['pink','green','blue','cyan','magenta'])

In [ ]:
data = df2
dummy =  pd.get_dummies(data["classfication"])

In [ ]:
#dummy

In [ ]:
data=data.drop(["classfication"],axis=1)

In [ ]:
data.shape

In [ ]:
data = data.join(dummy)

In [ ]:
data.shape

In [ ]:
#data.columns

In [ ]:
dummy =  pd.get_dummies(data["type1"])

In [ ]:
#dummy

In [ ]:
#data=data.drop(["type1"],axis=1)

In [ ]:
dummy2 =  pd.get_dummies(data["type2"])

In [ ]:
for i in dummy.columns:
    if i in dummy2.columns:
        for j in range(0,len(dummy.index)):
            dummy[i][j] = dummy[i][j] | dummy2[i][j]
        dummy2 = dummy2.drop([i],axis=1)

In [ ]:
data = data.join(dummy)

In [ ]:
data.shape

In [ ]:
data=data.drop(["type2"],axis=1)

In [ ]:
data=data.drop(["type1"],axis=1)

In [ ]:
ability_df = df2.filter(regex=("abilities_.*"))

In [ ]:
ability_df.columns[0]
dummy = pd.get_dummies(data[ability_df.columns[0]])
if "No" in dummy.columns:
    dummy.drop(["No"],axis=1)
for i in range(1,len(ability_df.columns)):
    dummy2 = pd.get_dummies(data[ability_df.columns[i]])
    if "No" in dummy2.columns:
        dummy2.drop(["No"],axis=1)
    for k in dummy2.columns:
        if k in dummy.columns:
            for j in range(0,len(dummy.index)):
                dummy[k][j] = dummy[k][j] | dummy2[k][j]
        else:
            dummy[k]=dummy2[k][:]

In [ ]:
#dummy2 =  pd.get_dummies(data["abilities_1"])
data = data.join(dummy)

In [ ]:
data.shape

In [ ]:
data=data.drop(ability_df.columns,axis=1)

In [ ]:
data.head()

In [ ]:
data=data.drop(["No"],axis=1)

In [ ]:
response=data["is_legendary"]
data=data.drop(["is_legendary"],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(data,response,test_size=0.2,random_state=0)

In [ ]:
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.fit_transform(x_test)

In [ ]:
x_test.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier= Sequential()

In [ ]:
classifier.add(Dense(units=10, kernel_initializer='uniform', activation='relu',input_dim = 867))
classifier.add(Dense(units=10,kernel_initializer='uniform',activation='relu'))
classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
classifier.fit(x_train,y_train,batch_size=10,epochs=100)

In [ ]:
pred = classifier.predict(x_test)

In [ ]:
y_pred = (pred > 0.5)

In [ ]:
accuracy_score(y_pred,y_test)